In [44]:
import torch
from PetDataset import PetDataset
import torchvision
from PIL import Image
import os
import torchvision.transforms as transforms

device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
print(device)

cuda:0


In [45]:
transform = transforms.Compose(
    [transforms.ToTensor(),
     transforms.Resize((32,32)),
     transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))])

dataset = PetDataset(data_transform=transform)
dataloader = torch.utils.data.DataLoader(dataset, batch_size=4)

dataiter = iter(dataloader)

images, labels = dataiter.next()

labels

tensor([0.6300, 0.4200, 0.2800, 0.1500], dtype=torch.float64)

In [46]:
import torch.nn as nn
import torch.nn.functional as F

class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.conv1 = nn.Conv2d(3, 6, 5)
        self.pool = nn.MaxPool2d(2, 2)
        self.conv2 = nn.Conv2d(6, 16, 5)
        self.fc1 = nn.Linear(16 * 5 * 5, 120)
        self.fc2 = nn.Linear(120, 84)
        self.fc3 = nn.Linear(84, 1)

    def forward(self, x):
        x = self.pool(F.relu(self.conv1(x)))
        x = self.pool(F.relu(self.conv2(x)))
        x = x.view(-1, 16 * 5 * 5)
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = torch.tanh(self.fc3(x))
        return x.view(-1)

In [ ]:
import torch.optim as optim
net = Net()
criterion = nn.MSELoss()
optimizer = optim.SGD(net.parameters(), lr=0.001, momentum=0.5)

for epoch in range(10):  # loop over the dataset multiple times

    running_loss = 0.0
    for i, data in enumerate(dataloader, 0):
        # get the inputs; data is a list of [inputs, labels]
        inputs, labels = data

        # zero the parameter gradients
        optimizer.zero_grad()

        # forward + backward + optimize
        outputs = net(inputs)
        loss = criterion(outputs, labels.float())
        loss.backward()
        optimizer.step()

        # print statistics
        running_loss += loss.item()
        if i % 200 == 199:    # print every 2000 mini-batches
            print('[%d, %5d] loss: %.3f' %
                  (epoch + 1, i + 1, running_loss / 200))
            running_loss = 0.0

print('Finished Training')


[1,   200] loss: 0.090
[1,   400] loss: 0.051
[1,   600] loss: 0.046
[1,   800] loss: 0.046
[1,  1000] loss: 0.038
[1,  1200] loss: 0.041
[1,  1400] loss: 0.047
[1,  1600] loss: 0.040
[1,  1800] loss: 0.046
[1,  2000] loss: 0.039
[1,  2200] loss: 0.042
[1,  2400] loss: 0.042
[2,   200] loss: 0.041
[2,   400] loss: 0.047
[2,   600] loss: 0.045
[2,   800] loss: 0.045
[2,  1000] loss: 0.038
[2,  1200] loss: 0.041
[2,  1400] loss: 0.047
[2,  1600] loss: 0.040
[2,  1800] loss: 0.045
[2,  2000] loss: 0.039
[2,  2200] loss: 0.042
[2,  2400] loss: 0.042
[3,   200] loss: 0.041
[3,   400] loss: 0.047
[3,   600] loss: 0.045
[3,   800] loss: 0.045
[3,  1000] loss: 0.038
[3,  1200] loss: 0.041
[3,  1400] loss: 0.047
[3,  1600] loss: 0.040
[3,  1800] loss: 0.045
[3,  2000] loss: 0.038
[3,  2200] loss: 0.042
[3,  2400] loss: 0.042
[4,   200] loss: 0.041
[4,   400] loss: 0.047
[4,   600] loss: 0.045
[4,   800] loss: 0.045
[4,  1000] loss: 0.038
[4,  1200] loss: 0.041
[4,  1400] loss: 0.047
[4,  1600] 

In [ ]:
correct = 0
total = 0
iter_i = 0

with torch.no_grad():
    for data in dataloader:
        images, labels = data
        outputs = net(images)

        if iter_i == 0:
            d = outputs
        else:
            d = torch.cat((d, outputs), 0)

        iter_i += 1

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

df = pd.read_csv("petfinder-pawpularity-score/train.csv")

plt.scatter(df["Pawpularity"], d.numpy(), alpha=0.4)